# PostgreSQL and AutoCommits

## Walk through the basics of PostgreSQL autocommits 

**Using pip command, you can install Psycopg2 on any operating system including Windows, macOS, Linux, and Unix and Ubuntu. Use the following pip command to install Psycopg2.**

In [ ]:
!pip install psycopg2

In [ ]:
## import postgreSQL adapter for the Python #others wrappers available? is python better, 
import psycopg2

In Python, we have several modules available to connect and work with PostgreSQL:

* Psycopg2
* py-postgresql
* pg8000
* ocpgdb
* PyGreSQL
* bpgsql

For this tutorial, we stick to the Psycopg2. Why?

* Psycopg2 is the most popular python driver for PostgreSQL.
* It is required for most Python and Postgres frameworks.
* Actively maintained and support the major version of python i.e. Python 3 and Python 2.
* It is thread-safe (threads can share the connections). It was designed for heavily multi-threaded applications

### Create a connection to the database

**Arguments required to connect PostgreSQL database from Python**

* The username you use to work with PostgreSQL, The default username for the PostgreSQL database is Postgres.
* Password – Password is given by the user at the time of installing the PostgreSQL.
* Host Name  – is the server name or Ip address on which PostgreSQL is running. if you are running on localhost, then you can use localhost, or it’s IP i.e., 127.0.0.0
* Database Name – Database name to which you want to connect. Here we are using Database named “postgres_db”.


1. Connect to the local instance of PostgreSQL (*127.0.0.1*)
2. Use the database/schema from the instance. 
3. The connection reaches out to the database (*Demodb*) and use the correct privilages to connect to the database (*user=postgres and password = 12345678*).

In [ ]:
conn = psycopg2.connect(user = "postgres",
                        password = "12345678",
                        host = "127.0.0.1",
                        port = "5432",
                        database = "demodb")

In [ ]:
#conn = psycopg2.connect("host=127.0.0.1 dbname=demodb user=postgres password=12345678")

### Use the connection to get a cursor that will be used to execute queries.

In [ ]:
cur = conn.cursor()

In [ ]:
# Print PostgreSQL Connection properties
print ( conn.get_dsn_parameters(),"\n")

{'user': 'postgres', 'dbname': 'demodb', 'host': '127.0.0.1', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 



### Create a database to work in

In [ ]:
cur.execute("select * from test")

UndefinedTable: relation "test" does not exist
LINE 1: select * from test
                      ^


### Error occurs, but it was to be expected because table has not been created as yet. To fix the error, create the table. 

In [ ]:
cur.execute("CREATE TABLE test (col1 int, col2 int, col3 int);")

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


### Error indicates we cannot execute this query. Since we have not committed the transaction and had an error in the transaction block, we are blocked until we restart the connection.

In [ ]:
conn = psycopg2.connect("host=127.0.0.1 dbname=demodb user=postgres password=12345678")
cur = conn.cursor()

In our exercises instead of worrying about commiting each transaction or getting a strange error when we hit something unexpected, let's set autocommit to true. **This says after each call during the session commit that one action and do not hold open the transaction for any other actions. One action = one transaction.**

In this demo we will use automatic commit so each action is commited without having to call `conn.commit()` after each command. **The ability to rollback and commit transactions are a feature of Relational Databases.**

In [ ]:
conn.set_session(autocommit=True)

In [ ]:
cur.execute("select * from test")

UndefinedTable: relation "test" does not exist
LINE 1: select * from test
                      ^


In [ ]:
cur.execute("CREATE TABLE test (col1 int, col2 int, col3 int);")

### Once autocommit is set to true, we execute this code successfully. There were no issues with transaction blocks and we did not need to restart our connection. 

In [ ]:
cur.execute("select * from test")

In [ ]:
cur.execute("select count(*) from test")
print(cur.fetchall())

[(0,)]


In [ ]:
cur.execute("drop table test")

In [ ]:
cur.close()
conn.close()